In [ ]:
print('hello')

In [2]:
#https://ithelp.ithome.com.tw/articles/10240314
#https://github.com/zhixuhao/unet?tab=readme-ov-file
import os
from PIL import Image,ImageOps
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

x_list = []
y_list = []

dir = "./LSCData/A1/"
for file in os.listdir(dir):
    if file.endswith("label.png"):
        y_list.append(np.array(Image.open(dir + file).convert('RGB')))
    else:
        x_list.append(np.array(Image.open(dir + file).convert('RGB')))

# print(x_list)
# print(y_list)
print(len(x_list))
print(len(y_list))



128
128


In [3]:
import matplotlib.pyplot as plt

# plt.imshow(x_list[0])
# plt.axis('off')
# plt.show()

# plt.imshow(y_list[0])
# plt.axis('off')
# plt.show()

In [4]:
train_x = np.array(x_list[:115])
train_y = np.array(y_list[:115])
val_x = np.array(x_list[115:])
val_y = np.array(y_list[115:])

# print(len(train_x))
# print(len(train_y))
# print(len(val_x))
# print(len(val_y))

print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(115, 530, 500, 3)
(115, 530, 500, 3)
(13, 530, 500, 3)
(13, 530, 500, 3)


In [5]:
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import layers

img_size = (530,500,3)
num_classes = 3

def get_model(img_size, num_classes):

    inputs = keras.Input(shape=img_size)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    conv1 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(inputs)
    conv1 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(conv1)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv1)

    conv2 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(pool1)
    conv2 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(conv2)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv2)

    conv3 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(pool2)
    conv3 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(conv3)#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    pool3 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv3)

    conv4 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(pool3)
    conv4 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(conv4)
    conv4_drop = Dropout(0.5)(conv4) #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    pool4 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv4_drop)
    conv5 = layers.Conv2D(1024, 3, padding="same" ,activation='relu')(pool4)
    conv5 = layers.Conv2D(1024, 3, padding="same" ,activation='relu')(conv5)
    conv5_drop = Dropout(0.5)(conv5)

    up1 = UpSampling2D(size = (2,2))(conv5_drop)
    up1 = layers.Conv2D(512, 2, padding="same" ,activation='relu')(up1)
    merge1 = concatenate([conv4_drop,up1], axis = 3)
    conv6 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(merge1)
    conv6 = layers.Conv2D(512, 3, padding="same" ,activation='relu')(conv6)

    up2 = UpSampling2D(size = (2,2))(conv6)
    up2 = layers.Conv2D(256, 2, padding="same" ,activation='relu')(up2)
    up2 = layers.ZeroPadding2D(padding=((0, 0), (0, 1)))(up2) #?????????? padding 
    merge2 = concatenate([conv3,up2], axis = 3)
    conv7 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(merge2)
    conv7 = layers.Conv2D(256, 3, padding="same" ,activation='relu')(conv7)

    up3 = UpSampling2D(size = (2,2))(conv7)
    up3 = layers.Conv2D(128, 2, padding="same" ,activation='relu')(up3)
    up3 = layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(up3) #?????????? padding 
    merge3 = concatenate([conv2,up3], axis = 3)
    conv8 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(merge3)
    conv8 = layers.Conv2D(128, 3, padding="same" ,activation='relu')(conv8)

    up4 = UpSampling2D(size = (2,2))(conv8)
    up4 = layers.Conv2D(64, 2, padding="same" ,activation='relu')(up4)
    merge4 = concatenate([conv1,up4], axis = 3)
    conv9 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(merge4)
    conv9 = layers.Conv2D(64, 3, padding="same" ,activation='relu')(conv9)
    conv9 = layers.Conv2D(2, 3, padding="same" ,activation='relu')(conv9)
    conv10 = layers.Conv2D(num_classes, 1, activation='sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = AdamW(), loss = 'E_crossentropy', metrics = ['accuracy'])
    model.summary()

    return model
    
model = get_model(img_size, num_classes)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 530, 500,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 530, 500,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 530, 500,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 265, 250,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 265, 250,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 265, 250,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 132, 125,  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 132, 125,  │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 132, 125,  │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 66, 62,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 66, 62,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 66, 62,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 66, 62,    │          0 │ conv2d_7[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 33, 31,    │          0 │ dropout[0][0]     │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 33, 31,    │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 33, 31,    │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 33, 31,    │          0 │ conv2d_9[0][0]  

 Total params: 31,032,843 (118.38 MB)

 Trainable params: 31,032,843 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

# 設定檢查點 callbacks，模型存檔
callbacks = [
    keras.callbacks.ModelCheckpoint("segmentation.keras", save_best_only=True)
]
# 訓練 15 週期(epoch)
epochs = 15
model.fit(train_x, train_y, verbose=1,batch_size=4, epochs=epochs, callbacks=callbacks,validation_data=(val_x, val_y))
loss, accuracy = model.evaluate(val_x, val_y, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/15
  1/115 ━━━━━━━━━━━━━━━━━━━━ 6:29:37 205s/step - loss: 18.7750